In [ ]:
import numpy as np
import pymc3 as pm3
import pymc as pm
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import optimize
import csv

In [ ]:
with open("data/data7a.csv") as f:
    reader = csv.reader(f)
    data = []
    next(reader)

    for row in reader:
        data.append(int(row[1]))
print(data)
data = np.array(data)

In [ ]:
N = len(data)

with pm3.Model() as model:
    s = pm3.Uniform("s", upper= 10**2, lower= 0)
    r = [pm3.Normal(name="r_{0}".format(i), mu=0., tau=s**-2) for i in range(N)]
    alpha = pm3.Uniform("alpha", upper= 10**2, lower= -(10**2))
    p = pm3.invlogit(alpha + r)
    obs = pm3.Binomial(name="obs", n=8, p=p, observed=data)
    
with model:
    start = pm3.find_MAP(fmin=optimize.fmin_powell)
    step = pm3.Metropolis()
    trace = pm3.sample(5000, start=start, step=step)
    pm3.summary(trace[2000:], varnames=["s", "alpha", "beta"])

In [ ]:
x = []
y = []

with open("data/data7.csv") as f:
    reader = csv.reader(f)
    next(reader)

    for row in reader:
        y.append(int(row[1]))
        x.append(int(row[2]))

x = np.array(x)
y = np.array(y)

In [ ]:
N = len(x)

with pm3.Model() as model:
    s = pm3.Uniform("s", upper= 10**2, lower= 0)
    r = [pm3.Normal(name="r_{0}".format(i), mu=0., tau=s**-2) for i in range(N)]
    alpha = pm3.Uniform("alpha", upper= 10**2, lower= -(10**2))
    beta =  pm3.Uniform("beta", upper= 10**2, lower= -(10**2))
    p = pm3.invlogit(alpha + r + beta * x)
    obs = pm3.Binomial(name="obs", n=8, p=p, observed=y)
    
with model:
    start = pm3.find_MAP(fmin=optimize.fmin_powell)
    #step = pm3.Metropolis()
    trace = pm3.sample(5000, start=start)
    pm3.summary(trace[2000:], varnames=["s", "alpha", "beta"])

In [ ]:
pm3.traceplot(trace[2000:], varnames=["s", "alpha", "beta"] )
plt.show()